### Import Libraries

In [2]:
# !pip install web3

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import pandas as pd
import numpy as np
from web3 import Web3
import requests
import os
import pytz
from datetime import datetime

### Define Variables

In [5]:
web3 = Web3(Web3.HTTPProvider(
    'https://mainnet.infura.io/v3/16b010cf4d3b4f4ba15f738554d10d67'))

ETHERSCAN_API_KEY = "UJSVWEU6FUMCE96JQ2UWNERHVW4YVP5PIE"

address_path = 'Data/Blocks_Address/15000000to15999999_address.csv'

path = '/content/drive/MyDrive/Colab Notebooks/'

export_file = 'Data/Dataset/data_normal_tnx_stats_82k.csv'

vietnam_timezone = pytz.timezone('Asia/Ho_Chi_Minh')

In [6]:
arbitrum_sequencer = "0xa4b10ac61e79ea1e150df70b8dda53391928fd14"
ethermine = "0xea674fdde714fd979de3edf0f56aa9716b898ec8"

In [7]:
# Define the array of values to exclude
values_to_exclude = [ethermine, arbitrum_sequencer]

### Define Functions

In [8]:
def query_txn_address(address, start_block=15000000, end_block=16999999):
	return f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock={start_block}&endblock={end_block}&page=1&offset=10000&sort=asc&apikey={ETHERSCAN_API_KEY}"

In [9]:
def query_txn_address_2(address, start_block=0):
	return f"https://api-sepolia.etherscan.io/api?module=account&action=txlist&address={address}&startblock={start_block}&endblock={19999999}&page=1&offset=10000&sort=asc&apikey={ETHERSCAN_API_KEY}"

In [10]:
def get_address_stats_normal_tnx(sample_df, address):
    sample_df['eth value'] = sample_df['value'].apply(
        lambda x: Web3.from_wei(int(x), 'ether'))

    # Typing of sent and received transactions
    sample_df['txn type'] = np.where(
        sample_df['from'].str.lower() == address.lower(), 'sent', 'received')

    sample_df.sort_values(by=['timeStamp'])
    sample_df['unix time difference'] = sample_df['timeStamp'].diff()
    sample_df_time_dim = sample_df.groupby('txn type')['unix time difference'].sum()/60

    # Handling of Sent transactions stats
    sample_df_sent = sample_df[sample_df['txn type'] == 'sent']
    if sample_df_sent.empty:
        core_stats_Sent_tnx = 0
        core_stats_MinValSent = 0
        core_stats_MaxValSent = 0
        core_stats_AvgValSent = 0
        core_stats_TotalEtherSent = 0
        core_stats_UniqueSentTo_Addresses = 0
        core_stats_Avg_min_between_sent_tnx = 0
        core_stats_AvgGasFee = 0
        core_stats_MinGasFee = 0
        core_stats_MaxGasFee = 0
    else:
        sample_df_sent = sample_df_sent.sort_values(by=['timeStamp'])
        sample_df_sent.loc[:, 'gas fee'] = sample_df_sent['gasPrice'] * \
            sample_df_sent['gasUsed']
        sample_df_sent['gas fee eth'] = sample_df_sent['gas fee'].apply(
            lambda x: Web3.from_wei(int(x), 'ether'))
        core_stats_MinGasFee = sample_df_sent['gas fee eth'].min()
        core_stats_MaxGasFee = sample_df_sent['gas fee eth'].max()
        core_stats_AvgGasFee = sample_df_sent['gas fee eth'].mean()
        core_stats_TotalGasFee = sample_df_sent['gas fee eth'].sum()
        core_stats_Sent_tnx = len(sample_df_sent)
        core_stats_MinValSent = sample_df_sent['eth value'].min()
        core_stats_MaxValSent = sample_df_sent['eth value'].max()
        core_stats_AvgValSent = sample_df_sent['eth value'].mean()


        core_stats_TotalEtherSent = sample_df_sent['eth value'].sum()
        core_stats_UniqueSentTo_Addresses = len(sample_df_sent['to'].unique())
        core_stats_Avg_min_between_sent_tnx = sample_df_time_dim['sent'] / \
            core_stats_Sent_tnx

    sample_df_received = sample_df[sample_df['txn type'] == 'received']
    # Handling of received transactions stats
    if sample_df_received.empty:
        core_stats_Received_tnx = 0
        core_stats_MinValueReceived = 0
        core_stats_MaxValueReceived = 0
        core_stats_AvgValueReceived = 0
        core_stats_TotalEtherReceived = 0
        core_stats_UniqueReceivedFrom_Addresses = 0
        core_stats_Avg_min_between_received_tnx = 0
    else:
        sample_df_received = sample_df_received.sort_values(by=['timeStamp'])
        core_stats_Received_tnx = len(sample_df_received)
        core_stats_MinValueReceived = sample_df_received['eth value'].min()
        core_stats_MaxValueReceived = sample_df_received['eth value'].max()
        core_stats_AvgValueReceived = sample_df_received['eth value'].mean()
        core_stats_TotalEtherReceived = sample_df_received['eth value'].sum()
        core_stats_UniqueReceivedFrom_Addresses = len(
            sample_df_received['from'].unique())
        core_stats_Avg_min_between_received_tnx = sample_df_time_dim['received'] / \
            core_stats_Received_tnx

    # Compilation of remaining normal transaction statistics
    core_stats_TimeDiffbetweenfirstand_last = (
        (sample_df['timeStamp'].max()) - (sample_df['timeStamp'].min()))/60
    core_stats_TotalTransactions = len(sample_df)
    core_stats_TotalEtherBalance = core_stats_TotalEtherReceived - core_stats_TotalEtherSent
    core_stats_NumberofError = len(sample_df[sample_df['isError'] != 0])

    compiled_normal_tnx_result = {
        'Address': address,
        'Avg min between sent tnx': core_stats_Avg_min_between_sent_tnx,
        'Avg min between received tnx': core_stats_Avg_min_between_received_tnx,
        'Time Diff between first and last (Mins)': core_stats_TimeDiffbetweenfirstand_last,
        'Sent tnx': core_stats_Sent_tnx,
        'Received Tnx': core_stats_Received_tnx,
        # 'Number of Created Contracts': core_stats_NumberofCreated_Contracts,
        'Number of Tnx Error': core_stats_NumberofError,
        'Unique Received From Addresses': core_stats_UniqueReceivedFrom_Addresses,
        'Unique Sent To Addresses': core_stats_UniqueSentTo_Addresses,
        'min value received': core_stats_MinValueReceived,
        'max value received ': core_stats_MaxValueReceived,
        'avg val received': core_stats_AvgValueReceived,
        'min val sent': core_stats_MinValSent,
        'max val sent': core_stats_MaxValSent,
        'avg val sent': core_stats_AvgValSent,
        # 'min value sent to contract': core_stats_MinValueSentToContract,
        # 'max val sent to contract': core_stats_MaxValueSentToContract,
        # 'avg value sent to contract': core_stats_AvgValueSentToContract,
        'min gas fee': core_stats_MinGasFee,
        'max gas fee': core_stats_MaxGasFee,
        'avg gas fee': core_stats_AvgGasFee,
        'total transactions (including tnx to create contract)': core_stats_TotalTransactions,
        'total Ether sent': core_stats_TotalEtherSent,
        'total ether received': core_stats_TotalEtherReceived,
        # 'total gas fee': core_stats_TotalGasFee,
        # 'total ether sent contracts': core_stats_TotalEtherSent_Contracts,
        'total ether balance': core_stats_TotalEtherBalance
    }

    return pd.DataFrame([compiled_normal_tnx_result])

In [11]:
def get_tx_by_address(address):
    try:
        # Fetch data from API
        res = requests.get(query_txn_address_2(address))
        res.raise_for_status()  # Raise an exception for 4XX and 5XX status codes
        data = res.json()['result']

        columns_to_keep = ['blockNumber', 'timeStamp', 'from', 'to', 'value', 'gas', 'gasPrice',
                           'isError', 'contractAddress', 'cumulativeGasUsed', 'gasUsed']
        columns_to_convert_int = ['blockNumber', 'timeStamp', 'isError']
        columns_to_convert_float = ['value', 'gasPrice', 'gas', 'cumulativeGasUsed', 'gasUsed']

        df = pd.DataFrame(data)[columns_to_keep]

        # Check if the total number of transactions returned is 10000
        if len(df) == 10000:
            # Perform another request from the last blockNumber to block 16999999
            last_block_number = df['blockNumber'].iloc[-1]
            new_res = requests.get(query_txn_address(address, last_block_number))
            new_res.raise_for_status()  # Raise an exception for 4XX and 5XX status codes
            new_data = new_res.json()['result']

            # Append the new data to the existing DataFrame
            new_df = pd.DataFrame(new_data)[columns_to_keep]
            df = pd.concat([df, new_df], ignore_index=True).drop_duplicates()

        # Convert columns to appropriate data types and handle missing values
        df.loc[:, columns_to_convert_int] = df[columns_to_convert_int].astype(np.int64).fillna(0)
        df.loc[:, columns_to_convert_float] = df[columns_to_convert_float].astype(np.float64).fillna(0)
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

In [12]:
def export_data(df):
    file_path = f'{path}{export_file}'
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode='a', newline='') as f:
        if not file_exists:
            df.to_csv(f, index=False)
        else:
            df.to_csv(f, index=False, header=False)

### Config

In [13]:
def processing_data():
    try:
        # Read address DataFrame and filter unwanted values
        address_df = pd.read_csv(f"{path}{address_path}", index_col=False)
        address_df_filtered = address_df[~address_df['Address'].isin(values_to_exclude)]

        address_df_filtered = address_df_filtered.iloc[82001:85001]  # Note: Row indexing starts from 0, so 517 corresponds to index 516
        count = 82001

        # Get transactions for each address in the DataFrame
        for address in address_df_filtered['Address']:
            current_time = datetime.now(vietnam_timezone)
            print(f"{current_time.strftime('%d-%m-%Y %H:%M:%S')} Processing {count} - {address}")
            df = get_tx_by_address(address)

            # Process the transaction data and export
            if not df.empty:
                sample = get_address_stats_normal_tnx(df, address)
                export_data(sample)

            count += 1
    except Exception as e:
        print(f"Error: {e}")

### Execution

In [14]:
address = "0x087791512bef6469b7ea2799a55d508a9bf6be33"

In [15]:
df = get_tx_by_address(address)
df

,blockNumber,timeStamp,from,to,value,gas,gasPrice,isError,contractAddress,cumulativeGasUsed,gasUsed
0,4084822,1691980116,0xa7e4ef0a9e15bdef215e2ed87ae050f974ecd60b,0x087791512bef6469b7ea2799a55d508a9bf6be33,500000000000000000.0,63000.0,723158595.0,0,,13261638.0,21000.0
1,5066990,1704988620,0x1fc35b79fb11ea7d4532da128dfa9db573c51b09,0x087791512bef6469b7ea2799a55d508a9bf6be33,500000000000000000.0,22000.0,70187429874.0,0,,5154056.0,21000.0
2,5073342,1705077684,0x087791512bef6469b7ea2799a55d508a9bf6be33,0x5e1341d31930496ea2c58c59e79b417e4ea57343,600000000000000000.0,21000.0,166995298044.0,0,,2110552.0,21000.0
3,5073351,1705077804,0x1fc35b79fb11ea7d4532da128dfa9db573c51b09,0x087791512bef6469b7ea2799a55d508a9bf6be33,500000000000000000.0,22000.0,137572849260.0,0,,3488773.0,21000.0
4,5073378,1705078164,0x6cc9397c3b38739dacbfaa68ead5f5d77ba5f455,0x087791512bef6469b7ea2799a55d508a9bf6be33,67853205000000000.0,21000.0,103994241481.0,0,,2192745.0,21000.0
5,5073380,1705078224,0x4281ecf07378ee595c564a59048801330f3084ee,0x087791512bef6469b7ea2799a55d508a9bf6be33,250000000000000000.0,60000.0,100385743849.0,0,,29200758.0,21000.0
6,5084450,1705233780,0x087791512bef6469b7ea2799a55d508a9bf6be33,0x8626f6940e2eb28930efb4cef49b2d1f2c9c1199,600000000000000000.0,21000.0,9607841191.0,0,,14139024.0,21000.0
7,5084530,1705234860,0x7ed746476a7f6520babd24eee1fdbcd0f7fb271f,0x087791512bef6469b7ea2799a55d508a9bf6be33,500000000000000000.0,63000.0,33427160385.0,0,,294000.0,21000.0
8,5084589,1705235772,0x087791512bef6469b7ea2799a55d508a9bf6be33,0x8626f6940e2eb28930efb4cef49b2d1f2c9c1199,400000000000000000.0,21000.0,19763544822.0,0,,8248337.0,21000.0
9,5113765,1705659948,0x3c352ea32dfbb757ccdf4b457e52daf6ecc21917,0x087791512bef6469b7ea2799a55d508a9bf6be33,500000000000000000.0,63000.0,219756422433.0,0,,42000.0,21000.0


In [16]:
sample = get_address_stats_normal_tnx(df, address)

In [17]:
sample

,Address,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Tnx Error,Unique Received From Addresses,Unique Sent To Addresses,min value received,...,min val sent,max val sent,avg val sent,min gas fee,max gas fee,avg gas fee,total transactions (including tnx to create contract),total Ether sent,total ether received,total ether balance
0,0x087791512bef6469b7ea2799a55d508a9bf6be33,1364.066667,40583.466667,369343.4,3,9,0,7,2,0.01,...,0.4,0.6,0.533333,0.000201764665011,0.003506901258924,0.001375,12,1.6,3.327853205,1.727853205


In [18]:
# processing_data()